# AG2: Tools (Tools Basics)

- Overview<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/tools/index
- Tools Basics<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/tools/basics

## Overview

Agents in AG2 leverage tools to extend their functionality, allowing them to interact with external systems, fetch real-time data, and execute complex tasks beyond the scope of a language model’s internal knowledge. This enables a structured approach where agents decide which tool to use and then execute it accordingly.
- How Tool Usage Works in AG2
  - **Selection**: An agent (driven by its LLM) decides which tool is appropriate based on the given task.
  - **Execution**: A separate executor agent invokes the tool and returns the results.
- Secure Tool Usage with Secrets
  - **Enhanced Security**: Keeps secrets out of LLM interactions and telemetry.
  - **Simplified Development**: Securely passes credentials without hardcoding them.
  - **Flexible Integration**: Allows agents to access necessary credentials while maintaining security.
- Interoperability with External Frameworks
  - **LangChain**: Provides numerous pre-built tools for API calls, web scraping, and more.
  - **CrewAI**: Offers specialized tools for web scraping, search, and automation.
  - **PydanticAI**: Supports structured data handling and dependency injection for context-driven tool execution.

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Tools Basics

In [2]:
from datetime import datetime
from typing import Annotated

# from autogen import ConversableAgent, register_function, LLMConfig
from autogen import ConversableAgent, register_function

# Put your key in the OPENAI_API_KEY environment variable
# llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")
llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}

# 1. Our tool, returns the day of the week for a given date
def get_weekday(date_string: Annotated[str, "Format: YYYY-MM-DD"]) -> str:
    date = datetime.strptime(date_string, "%Y-%m-%d")
    return date.strftime("%A")


# 2. Agent for determining whether to run the tool
date_agent = ConversableAgent(
    name="date_agent",
    system_message="You get the day of the week for a given date.",
    llm_config=llm_config,
)

# 3. And an agent for executing the tool
executor_agent = ConversableAgent(
    name="executor_agent",
    human_input_mode="NEVER",
)

# 4. Registers the tool with the agents, the description will be used by the LLM
register_function(
    get_weekday,
    caller=date_agent,
    executor=executor_agent,
    description="Get the day of the week for a given date",
)

In [3]:
# 5. Two-way chat ensures the executor agent follows the suggesting agent
chat_result = executor_agent.initiate_chat(
    recipient=date_agent,
    message="I was born on the 25th of March 1995, what day was it?",
    max_turns=2,
)

executor_agent (to date_agent):

I was born on the 25th of March 1995, what day was it?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
date_agent (to executor_agent):

***** Suggested tool call (call_f6c5kivGyvOVGw6aHxmVu72z): get_weekday *****
Arguments: 
{"date_string":"1995-03-25"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_weekday...
Call ID: call_f6c5kivGyvOVGw6aHxmVu72z
Input arguments: {'date_string': '1995-03-25'}
executor_agent (to date_agent):

***** Response from calling tool (call_f6c5kivGyvOVGw6aHxmVu72z) *****
Saturday
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
date_agent (to executor_agent):

You were born on a Saturday, th

In [4]:
print(chat_result.chat_history[-1]["content"])

You were born on a Saturday, the 25th of March 1995.


### decorators

In [5]:
@date_agent.register_for_llm(description="Get the day of the week for a given date")
@executor_agent.register_for_execution()
def get_weekday(date_string: Annotated[str, "Format: YYYY-MM-DD"]) -> str:
    date = datetime.strptime(date_string, '%Y-%m-%d')
    return date.strftime('%A')

C:\Users\Pavel\projects\ai-llm-agents\.venv\Lib\site-packages\autogen\agentchat\conversable_agent.py:2911: UserWarning: Function 'get_weekday' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)
C:\Users\Pavel\projects\ai-llm-agents\.venv\Lib\site-packages\autogen\agentchat\conversable_agent.py:3002: UserWarning: Function 'get_weekday' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)


In [6]:
@date_agent.register_for_llm(description="Get the day of the week for a given date")
@executor_agent.register_for_execution()
def get_weekday_2(date_string: Annotated[str, "Format: YYYY-MM-DD"]) -> str:
    date = datetime.strptime(date_string, '%Y-%m-%d')
    return date.strftime('%A')